# Purpose of this notebook is to generate model activation heatmaps video for all duration of model training

## Setup the constants and run whole notebook

In [14]:
DATA_ROOT = "/home/gpu1/Desktop/kamilio/Pneumothorax"
dataset_version = "data_v4.0.0"
split_type = "train"
IMG_SIZE = 256

MODEL_ROOT = "/home/gpu1/Desktop/kamilio/models"
model_category = "SSL_v10"
model_version = "model_v10.3.0"

# Set "activation_heatmaps" or "feature_embeddings" ("feature embeddings"  to be added)
type_of_video = "feature_embeddings"

# if type_of_video=="activation_heatmaps" specify
images_per_class = 5

## Start of code

### Loading necessary dependencies

In [15]:
import os
import time
import glob
import numpy as np
import pandas as pd
import cv2
from sklearn.decomposition import PCA
from sklearn.cluster import SpectralClustering
from sklearn.metrics import normalized_mutual_info_score
from matplotlib import pyplot as plt
from matplotlib import cm
import tensorflow as tf

from __future__ import print_function  # for Python2
import sys

from tf_keras_vis.gradcam import Gradcam
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
replace2linear = ReplaceToLinear()
from tf_keras_vis.utils.scores import CategoricalScore

### Creating path variables

In [16]:
path_to_model = os.path.join(MODEL_ROOT, model_category, model_version)
path_to_model_checkpoints = os.path.join(MODEL_ROOT, model_category, model_version, "checkpoints")
if type_of_video == "activation_heatmaps":
    path_to_video = os.path.join(MODEL_ROOT, model_category, model_version, "activation_heatmaps")
    path_to_video_frames = os.path.join(MODEL_ROOT, model_category, model_version, "activation_heatmaps", "frames")
elif type_of_video == "feature_embeddings":
    path_to_video = os.path.join(MODEL_ROOT, model_category, model_version, "feature_embeddings")
    path_to_video_frames = os.path.join(MODEL_ROOT, model_category, model_version, "feature_embeddings", "frames")

In [17]:
# files = os.listdir(path_to_model_checkpoints)
# files1 = [file for file in files if file.endswith(".index")]
# files2 = [file for file in files if file.endswith("001")]
# for num, file in enumerate(files2):
#     file_new = file[:12] + str(int(num)+501) + file[15:]
#     src = os.path.join(path_to_model_checkpoints, file)
#     dst = os.path.join(path_to_model_checkpoints, file_new)
#     os.rename(src, dst)

### Creating directories for frames and video 

In [18]:
os.makedirs(path_to_video)
os.makedirs(path_to_video_frames)

### Loading necessary functions

In [19]:
import os
import albumentations as A
AUTOTUNE = tf.data.AUTOTUNE
from functools import partial

transforms = A.Compose([
])

def get_classes(dataset_name):
    #TODO add functionality to pop error if classes in data splits do not match
    class_names = os.listdir(os.path.join(DATA_ROOT, dataset_name, "train"))
    class_names = [int(name) for name in class_names]
    class_names.sort()
    class_names = [str(name) for name in class_names]
    print(class_names)
    return class_names


def get_classes_dict(dataset_name):
    class_names = get_classes(dataset_name)
    class_names_dict = {}
    for class_name in class_names:
        image_sample_name = os.path.basename(glob.glob(os.path.join(DATA_ROOT, dataset_name, "train", str(class_name),'*'))[0])
        class_names_dict[str(class_name)] = image_sample_name.split('_')[-2]
    return class_names_dict

def get_classes_dict_empty_lists(dataset_name):
    class_names = get_classes(dataset_name)
    class_names_dict = {}
    for class_name in class_names:
        class_names_dict[str(class_name)] = []
    return class_names_dict

def get_label(file_path):
    # Convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    # The second to last is the class-directory
    # TODO: pass CLASS_NAMES as function parameter
    one_hot = parts[-2] == CLASS_NAMES
    # Integer encode the label
    return int(one_hot)

def decode_img(img):
    # Convert the compressed string to a 3D uint8 tensor
    img = tf.io.decode_jpeg(img, channels=3)
    return img

def aug_fn(image):
    data = {"image":image}
    aug_data = transforms(**data)
    aug_img = aug_data["image"]
    # Resize and norm the image
    aug_img = tf.image.resize(aug_img, size=[IMG_SIZE, IMG_SIZE])
    return aug_img

def apply_transformations(image):
    aug_img = tf.numpy_function(func=aug_fn, inp=[image], Tout=tf.float32)
    return aug_img

def process_path(file_path, use_augmentations: bool):
    label = get_label(file_path)
    # Load the raw data from the file as a string
    image = tf.io.read_file(file_path)
    image = decode_img(image)
    image = tf.cast(image/255, tf.float32)
    if use_augmentations:
        image = apply_transformations(image)
    return image, label

def load_dataset(dataset_name:str, split_type: str, use_transformations: bool, mini_batch_size: int):
    global CLASS_NAMES
    CLASS_NAMES = get_classes(dataset_name)
    dataset = tf.data.Dataset.list_files(f"{DATA_ROOT}/{dataset_name}/{split_type}/*/*", shuffle=False)
    dataset = dataset.shuffle(len(dataset), reshuffle_each_iteration=False)
    dataset = dataset.map(partial(process_path, use_augmentations=use_transformations), num_parallel_calls=AUTOTUNE).batch(mini_batch_size)
    return dataset

In [20]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras import layers

def get_initial_weights_type(imagenet_pretrained_backbone: bool):
    if imagenet_pretrained_backbone:
        weights='imagenet'
    else:
        weights=None
    return weights

def rebuild_top(model, class_count: int, top_dropout_rate: float = 0):
    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization(name='batch_normalization')(x)
    if top_dropout_rate != 0:
        x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(class_count, activation="softmax", name="pred")(x)
    return outputs

def build_EffNetB0(class_count: int, imagenet_pretrained_backbone: bool, top_dropout_rate: float = 0):
    
    weights_type = get_initial_weights_type(imagenet_pretrained_backbone)
    
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    model = EfficientNetB0(include_top=False, input_tensor=inputs, weights=weights_type)
    outputs = rebuild_top(model, class_count, top_dropout_rate)

    model = tf.keras.Model(inputs, outputs, name='EfficientNetB0')
    print("EffNet0 model build successfull")
    
    return model

def build_EffNetB3(class_count: int, imagenet_pretrained_backbone: bool, top_dropout_rate: float = 0):
    
    weights_type = get_initial_weights_type(imagenet_pretrained_backbone)
    
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    model = EfficientNetB3(include_top=False, input_tensor=inputs, weights=weights_type)
    outputs = rebuild_top(model, class_count, top_dropout_rate)

    model = tf.keras.Model(inputs, outputs, name='EfficientNetB3')
    print("EffNet3 model build successfull")
    
    return model

def build_EffNetB5(class_count: int, imagenet_pretrained_backbone: bool, top_dropout_rate: float = 0):
    
    weights_type = get_initial_weights_type(imagenet_pretrained_backbone)
    
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    model = EfficientNetB5(include_top=False, input_tensor=inputs, weights=weights_type)
    outputs = rebuild_top(model, class_count, top_dropout_rate)

    model = tf.keras.Model(inputs, outputs, name='EfficientNetB5')
    print("EffNet5 model build successfull")
    
    return model

def build_model(class_count: int, imagenet_pretrained_backbone: bool, architecture: str,  top_dropout_rate: float = 0):
    if architecture == "EffNetB0":
        model = build_EffNetB0(class_count, imagenet_pretrained_backbone, top_dropout_rate)
    elif architecture == "EffNetB3":
        model = build_EffNetB3(class_count, imagenet_pretrained_backbone, top_dropout_rate)
    elif architecture == "EffNetB5":
        model = build_EffNetB5(class_count, imagenet_pretrained_backbone, top_dropout_rate)
    else:
        raise Exception(f"Specified model architecture is not available")
        
    return model

### Loading the dataset

In [ ]:
map_dict = get_classes_dict(dataset_version)
sample_dict = get_classes_dict_empty_lists(dataset_version)
class_count = len(get_classes(dataset_version))

In [ ]:
if type_of_video == "feature_embeddings":
    ds = load_dataset(
                      dataset_name = dataset_version, 
                      split_type = split_type, 
                      use_transformations = False, 
                      mini_batch_size = 64)
    
    y_true = np.concatenate([y for x, y in ds], axis=0)
    y_true = [str(y[1]) for y in y_true]
    y_true_int = np.array([int(y) for y in y_true])

In [23]:
if type_of_video == "activation_heatmaps":
    ds = load_dataset(
                      dataset_name = dataset_version, 
                      split_type = split_type, 
                      use_transformations = False, 
                      mini_batch_size = 1)
    iterator = iter(ds)
    
    for _ in range(len(ds)):
        sample = next(iterator)
        image = sample[0].numpy().reshape(IMG_SIZE,IMG_SIZE,3)
        label = tf.math.argmax(sample[1], axis=1).numpy()[0]
        sample_dict[str(label)].append(image)

### Generating frames

In [ ]:
tf.keras.backend.clear_session()
model = tf.keras.models.load_model(path_to_model)
print(model_version)
print('\n')
checkpoints = os.listdir(path_to_model_checkpoints)
checkpoints = [f"{x.split('.')[0]}.ckpt" for x in checkpoints if x.endswith(".index")]
epoch_nmi = []

# checkpoints = ["weights00000001.ckpt","weights00000138.ckpt","weights00000500.ckpt"]




if type_of_video == "activation_heatmaps":
    print("Generating model activation heatmap frames")
elif type_of_video == "feature_embeddings":
    print("Generating model feature embeddings frames")

for checkpoint in checkpoints:
    epoch = int(checkpoint.split('.')[0][-4:])
    start = time.time()
    model.load_weights(os.path.join(path_to_model_checkpoints, checkpoint))
    print(f"checkpoint: {checkpoint}")

    if type_of_video == "activation_heatmaps":

        plt.figure(figsize=(3*class_count, 3.5*images_per_class))
#         plt.suptitle(f"Activation heatmaps of {model_version}. Checkpoint: {checkpoint.split('.')[0]}", fontsize=24)
        plt.suptitle(f"Checkpoint: {checkpoint.split('.')[0]}", fontsize=24)
        i=1

        # Create GradCAM++ object
        gradcam = Gradcam(model,
                                  model_modifier=replace2linear,
                                  clone=True)

        for row in range(images_per_class):
            for column in range(class_count):
                # Generate heatmap with GradCAM++
                cam = gradcam(CategoricalScore([int(column)]),
                              [sample_dict[str(column)][row]],
                              penultimate_layer=-1)
                heatmap = np.uint8(cm.jet(cam[0])[..., :3] * 255).reshape(IMG_SIZE,IMG_SIZE,3)

                plt.subplot(images_per_class, class_count, i)
                image = sample_dict[str(column)][row]
                classs = map_dict[str(column)].replace('-', ',')
                plt.imshow(image)
                plt.imshow(heatmap, cmap='jet', alpha=0.5)
                if row == 0:
                    plt.title(f'Degrees: {classs}', fontsize=16)
                if not(row==0 and column==0):
                    plt.axis('off')
                i += 1
        plot_name =  f"{checkpoint.split('.')[0]}.jpg"
        plt.savefig(os.path.join(path_to_video_frames, plot_name))
        plt.close()
        print(f"Saved image:  {plot_name}\n")
        end = time.time()
        print(f"Duration: {round(end-start, 2)} seconds\n")

    elif type_of_video == "feature_embeddings":
        # Reinitializing dense classification layer
        inputs = model.input
        x = model.layers[-5].output
        x_GAP = layers.GlobalAveragePooling2D(name="avg_pool")(x)

        model_GAP = tf.keras.Model(inputs, x_GAP)

        features = np.array(model_GAP.predict(ds, verbose = 0))
        features = features.T
        print('Fitting PCA')
        pca = PCA()
        pca.fit(features)

        first_component = pca.components_[0,:]
        second_component = pca.components_[1,:]
        PCA_first_two_components = np.concatenate((first_component.reshape(-1, 1), second_component.reshape(-1, 1)), axis=1)
        SpectralClustering_model = SpectralClustering(n_clusters=2)
        SpectralClustering_pred = SpectralClustering_model.fit_predict(pca.components_[0:10,:].T)
        nmi = normalized_mutual_info_score(y_true_int, SpectralClustering_pred)
        nmi_rounded = round(nmi, 5)

        print(f"NMI: {str(nmi_rounded)}")



        df = pd.DataFrame({"first": first_component, "second": second_component, "y_true": y_true, "SpectralClustering_pred": [str(y) for y in SpectralClustering_pred]})
        class_1_count = len(df["SpectralClustering_pred"][df["SpectralClustering_pred"]=='1'])
        epoch_nmi.append([epoch, nmi_rounded])
        colors = {"0": "red", "1": "green"}

        # Create two subplots side by side
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

        # Plot y_true in the first subplot
        ax1.scatter(df['first'], df['second'], c=df['y_true'].map(colors), s=1.5)
        ax1.set_title(f"True labels")
        ax1.set_xlabel('first component')
        ax1.set_ylabel('second component')
        ax1.legend()

        # Plot DBSCAN_pred in the second subplot
        ax2.scatter(df['first'], df['second'],c=df['SpectralClustering_pred'].map(colors), s=1.5)
        ax2.set_title(f"SpectralClustering. First 10 PCA components. (NMI: {str(nmi_rounded)})")
        ax2.set_xlabel('first component')
        ax2.set_ylabel('second component')
        ax2.legend()

        # Add a shared title to the subplots
        fig.suptitle(f"{model_version} feature embeddings. First two components of PCA. Checkpoint: {checkpoint.split('.')[0]}")


        # Save the plot and close the figure
        plot_name = f"{checkpoint.split('.')[0]}.jpg"
        plt.savefig(os.path.join(path_to_video_frames, plot_name))
        plt.close()
        print(f"Saved image: {plot_name}")

#         colors = {"0":'red', "1":'green'}
#         plt.scatter(df['first'], df['second'], c=df['label'].map(colors), s=1.5)
#         plt.title(f"{model_version} feature embeddings. First two components of PCA.\n Checkpoint: {checkpoint.split('.')[0]}. NMI:{str(nmi_rounded)}")
#         plt.xlabel('first component')
#         plt.ylabel('second component')
#         plot_name =  f"{checkpoint.split('.')[0]}.jpg"
#         plt.savefig(os.path.join(path_to_video_frames, plot_name))
#         plt.close()
#         print(f"Saved image:  {plot_name}")
        end = time.time()
        print(f"Duration: {round(end-start, 2)} seconds\n")

if type_of_video == "feature_embeddings":
    nmi_title =  f"NMI_{model_version}_{dataset_version}_{split_type}"
    epoch_nmi = np.array(epoch_nmi)
    epoch_nmi_df = pd.DataFrame({"Epoch": epoch_nmi[:,0],"NMI": epoch_nmi[:,1]})
    max_row_index = epoch_nmi_df["NMI"].idxmax()
    max_row = epoch_nmi_df.iloc[max_row_index]
    epoch_nmi_df.plot(x="Epoch", y="NMI", kind="line")
    epoch_nmi_df.to_csv(os.path.join(path_to_model, f"{nmi_title}_SpectralClustering_test.csv"))

    # Add axis labels and a title
    plt.xlabel("Epoch")
    plt.ylabel("NMI")
    plt.grid()
    plt.title(f"NMI vs Epoch. Best epoch: {str(int(max_row[0]))} (NMI={str(max_row[1])})")

    # Show the plot
    plt.savefig(os.path.join(path_to_model, f"{nmi_title}.jpg"))


### Generating video

In [25]:
# Get all the filenames of the frames in the directory
frame_filenames = os.listdir(path_to_video_frames)

# Sort the filenames in ascending order
frame_filenames.sort()

# Get the first frame to get its size
frame = cv2.imread(os.path.join(path_to_video_frames, frame_filenames[0]))
height, width, channels = frame.shape

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(os.path.join(path_to_video, "video.mp4"), fourcc, 5.0, (width, height))

# Loop through all the frames and write them to the output video
for filename in frame_filenames:
    frame_path = os.path.join(path_to_video_frames, filename)
    frame = cv2.imread(frame_path)
    out.write(frame)

# Release everything if job is finished
out.release()
cv2.destroyAllWindows()

In [ ]:
tf.keras.backend.clear_session()
start = time.time()
model = build_model(
                    class_count = 2, 
                    imagenet_pretrained_backbone = True,
                    architecture = 'EffNetB0',
                    top_dropout_rate = 0.2)
inputs = model.input
x = model.layers[-5].output
x_GAP = layers.GlobalAveragePooling2D(name="avg_pool")(x)

model_GAP = tf.keras.Model(inputs, x_GAP)

features = np.array(model_GAP.predict(ds, verbose = 0))
features = features.T
print('Fitting PCA')
pca = PCA()
pca.fit(features)

first_component = pca.components_[0,:]
second_component = pca.components_[1,:]
PCA_first_two_components = np.concatenate((first_component.reshape(-1, 1), second_component.reshape(-1, 1)), axis=1)
SpectralClustering_model = SpectralClustering(n_clusters=2)
SpectralClustering_pred = SpectralClustering_model.fit_predict(pca.components_[0:10,:].T)
nmi = normalized_mutual_info_score(y_true_int, SpectralClustering_pred)
nmi_rounded = round(nmi, 5)

print(f"NMI: {str(nmi_rounded)}")



df = pd.DataFrame({"first": first_component, "second": second_component, "y_true": y_true, "SpectralClustering_pred": [str(y) for y in SpectralClustering_pred]})
class_1_count = len(df["SpectralClustering_pred"][df["SpectralClustering_pred"]=='1'])
colors = {"0": "red", "1": "green"}

# Create two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

# Plot y_true in the first subplot
ax1.scatter(df['first'], df['second'], c=df['y_true'].map(colors), s=1.5, label=['pos', 'neg'])
ax1.set_title(f"True labels")
ax1.set_xlabel('first component')
ax1.set_ylabel('second component')
ax1.legend()

# Plot DBSCAN_pred in the second subplot
ax2.scatter(df['first'], df['second'],c=df['SpectralClustering_pred'].map(colors), s=1.5)
ax2.set_title(f"SpectralClustering. First 10 PCA components. (NMI: {str(nmi_rounded)})")
ax2.set_xlabel('first component')
ax2.set_ylabel('second component')
ax2.legend()

# Add a shared title to the subplots
fig.suptitle(f"ImageNet model feature embeddings. First two components of PCA.")


# Save the plot and close the figure
plot_name =  f"ImageNet.jpg"
plt.savefig(os.path.join(path_to_video_frames, plot_name))
plt.close()
print(f"Saved image:  {plot_name}")
end = time.time()
print(f"Duration: {round(end-start, 2)} seconds\n")
